# Métodos iterativos creados a mano

In [1]:
using LinearAlgebra, SparseArrays, BenchmarkTools
#using CUDA, CUDA.CUSPARSE

## Jacobi secuencial

### Matricial

In [2]:
function vectorizedjacobi(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= D⁻¹(L+U)x⁽ᵏ⁻¹⁾+D⁻¹b
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    # Definimos a N como la suma de estas dos últimas matrices.
    D = Diagonal(A)
    N = - triu(A,1) - tril(A,-1)
    x = rand(length(b))
    while norm(A*x - b) > ϵ
        x = D\(N*x + b)
    end
    return Vector(x)
end

vectorizedjacobi (generic function with 2 methods)

### Paralelo

In [3]:
function paralleljacobi(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    D = diag(A);
    D = CuArray{Float64}(D)
    N = - triu(A,1) - tril(A,-1)
    N = CuSparseMatrixCSR{Float64}(N)
    
    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    while norm(A*x - b) > ϵ
        x = D\(N*x + b)
    end
    return x
end

paralleljacobi (generic function with 2 methods)

---

## Gauss-Seidel

### Matricial

In [4]:
function vectorizedgaussseidel(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    U = - triu(A,1) 
    L₀ = tril(A,0)
    x = rand(length(b))
    while norm(A*x - b) > ϵ
        x = L₀\(U*x + b)
    end
    return x
end

vectorizedgaussseidel (generic function with 2 methods)

### Paralelo

In [5]:
function parallelgaussseidel(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    U = - triu(A,1) 
    L₀ = tril(A,0)
    U = CuSparseMatrixCSR{Float64}(U)
    L₀ = CuSparseMatrixCSR{Float64}(L₀)

    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    while norm(A*x - b) > ϵ
        x = L₀\(U*x + b)
    end
    return x
end

parallelgaussseidel (generic function with 2 methods)

---

## SOR (Successive Overrelaxation)

### Matricial

In [6]:
function vectorizedsor(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ω = 1.8::Float64, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-ωL)⁻¹(ωU+(1-ω)D)x⁽ᵏ⁻¹⁾+ω(D-ωL)⁻¹b
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    # ω es justo un argumento de sobrerelajación, que sirve para hacer más rápido al método. Hay que escogerlo 
    # ente (0,1)
    D = Diagonal(A)
    U = - triu(A,1) 
    L = - tril(A,-1)
    x = rand(length(b))
    
    L′ = D-ω*L
    U′ = ω*U+(1.0-ω)*D
    b′ = ω*b
    while norm(A*x - b) > ϵ
        x = L′ \ (U′*x + b′)
    end
    return x
end

vectorizedsor (generic function with 3 methods)

### Paralelo

In [7]:
function parallelsor(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ω = 1.8::Float64, ϵ = 1e-5)
    # Según el libro, viene siendo así el método matricial: x⁽ᵏ⁾= (D-L)⁻¹(Ux⁽ᵏ⁻¹⁾+b)
    # Con D := La matriz diagonal A como matriz cuadrada,
    # -L y -U las matrices estríctamente Inferior (Lower) y Superior (Upper) de A, respectivamente
    D = Diagonal(A)
    D = CuArray{Float64}(D)
    U = - triu(A,1) 
    L = - tril(A,-1)
    U = CuSparseMatrixCSR{Float64}(U)
    L = CuSparseMatrixCSR{Float64}(L)

    A = CuSparseMatrixCSR{Float64}(A)
    b = CuArray{Float64}(b)
    x = CUDA.rand{Float64}(length(b))
    
    L′ = D-ω*L
    U′ = ω*U+(1.0-ω)*D
    b′ = ω*b
    while norm(A*x - b) > ϵ
        x = L′ \ (U′*x + b′)
    end
    return x
end

parallelsor (generic function with 3 methods)

---

## GMRES

### Normalito

In [8]:
function leastsquares(H::Matrix{Float64}, r)
    r′ = zeros(size(H)[1])
    r′[1] = norm(r)
    x = H \ r′
end

leastsquares (generic function with 1 method)

In [9]:
# Algoritmo sacado directo del pseudocódigo
function gmres(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
        end
        k += 1
    end
    return x
end

gmres (generic function with 2 methods)

### Normalito GPU

In [10]:
# Algoritmo sacado directo del pseudocódigo
function parallelgmres(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
        end
        k += 1
    end
    return x
end

parallelgmres (generic function with 2 methods)

### Reiniciado

In [39]:
function reiniciarvariables(x::Vector, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64})
    k = 1
    x₀ = x
    r = b - A*x₀
    H = zeros(2,1)
    q = [r / norm(r)]
    return x₀, r, q, k, H
end
    
function gmresreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, mod, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        #if k % 25 == 0 # cuando tenemos 30³ nuestra matriz ( Q = 5,400,000)
        if k % mod == 0 # cuando tenemos 40³ nuestra matriz ( Q = 10,240,000)
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            x₀, residual₀, q, k, H = reiniciarvariables(x, A, b)
        end
    end
    return x
end

gmresreiniciado (generic function with 3 methods)

### Reiniciado GPU

In [12]:
function reiniciarvariables(x, A, b)
    k = 1
    x₀ = x
    r = b - A*x₀
    H = zeros(2,1)
    q = [r / norm(r)]
    return x₀, r, q, k, H
end
    
function parallelgmresreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = b - A*x₀
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        y = A*q[k]
        for j ∈ 1:k
            H[j,k] = q[j]' * y
            y -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(y)
        push!(q, y/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        #if k % 25 == 0 # cuando tenemos 30³ nuestra matriz ( Q = 5,400,000)
        if k % 25 == 0 # cuando tenemos 40³ nuestra matriz ( Q = 10,240,000)
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            x₀, residual₀, q, k, H = reiniciarvariables(x, A, b)
        end
    end
    return x
end

parallelgmresreiniciado (generic function with 2 methods)

### Precondicionado

In [13]:
function gmresprecondicionado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = (b - A*x₀)
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
        end
        k += 1
    end
    return x
end

gmresprecondicionado (generic function with 2 methods)

### Precondicionado GPU

In [14]:
function parallelgmresprecondicionado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = (b - A*x₀)
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        if k % 10 == 0
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
        end
        k += 1
    end
    return x
end

parallelgmresprecondicionado (generic function with 2 methods)

### Precondicionado y reiniciado

In [40]:
function reiniciarvariablesprecondicionado(x::Vector, A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        Ms::Vector)
    k = 1
    x₀ = x
    H = zeros(2,1) 
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    return x₀, residual₀, q, k, H
end

# Algoritmo sacado directo del pseudocódigo
function gmresprecondicionadoreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, mod, 
        ϵ = 1e-5)
    x₀ = rand(length(b))
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        # if k % 20 == 0 # Para 40³ sirvió chido el módulo 20 para el de Jacobi
        # if k % 25 == 0 # Para 40³ sirvió chido el módulo 25 para el de Gauss-Seidel
        if k % mod == 0 # Para 40³ sirvió chido el módulo 30 para el de SOR
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
            x₀, residual₀, q, k, H = reiniciarvariablesprecondicionado(x, A, b, Ms)
        end
    end
    return x
end

gmresprecondicionadoreiniciado (generic function with 3 methods)

### Precondicionado y reiniciado GPU

In [16]:
function reiniciarvariablesprecondicionado(x, A, b, Ms::Vector)
    k = 1
    x₀ = x
    H = zeros(2,1)
    
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    return x₀, residual₀, q, k, H
end

# Algoritmo sacado directo del pseudocódigo
function parallelgmresprecondicionadoreiniciado(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, Ms::Vector, 
        ϵ = 1e-5)
    b = CuArray{Float64}(b)
    A = CuSparseMatrixCSR{Float64}(A)
    x₀ = CUDA.rand{Float64}(length(b))
    
    residual₀ = b - A*x₀
    [residual₀ = M\residual₀ for M ∈ Ms]
    q = [residual₀ / norm(residual₀)]
    
    k = 1
    x = x₀
    H = zeros(2,1)
    residual = residual₀
    while norm(residual) > ϵ
        ω = A*q[k]
        [ω = M\ω for M ∈ Ms]
        for j ∈ 1:k
            H[j,k] = q[j]' * ω
            ω -= H[j,k]*q[j]
        end
        H[k+1,k] = norm(ω)
        push!(q, ω/H[k+1,k])
        H = vcat(H, zeros(1, size(H)[2]))
        H = hcat(H, zeros(size(H)[1], 1))
        k += 1
        # if k % 20 == 0 # Para 40³ sirvió chido el módulo 20 para el de Jacobi
        # if k % 25 == 0 # Para 40³ sirvió chido el módulo 25 para el de Gauss-Seidel
        if k % 30 == 0 # Para 40³ sirvió chido el módulo 30 para el de SOR
            c = leastsquares(H, residual₀)
            Q = hcat(q...)
            c = CuArray{Float64}(c)
            Q = CuSparseMatrixCSR{Float64}(Q)
            x = Q*c + x₀
            residual = A*x - b
            [residual = M\residual for M ∈ Ms]
            x₀, residual₀, q, k, H = reiniciarvariablesprecondicionado(x, A, b, Ms)
        end
    end
    return x
end

parallelgmresprecondicionadoreiniciado (generic function with 2 methods)

---

In [17]:
include("../notebooks/FVM.jl")

function Γ_constant(x::Array, y::Array, z::Array)
    Γ = 1000
    tensor_Γ = ones(length(x), length(y), length(z))
    tensor_Γ = Γ .* tensor_Γ
end

function getsystem(vol)
    # Construcción de la malla.
    volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces = FVM.uniform_grid(vol, vol/10)
    tags = FVM.init_tags(3, volumes, centers_and_boundaries)
    tags_b = FVM.init_tags_boundaries(3, centers_and_boundaries)
    FVM.tag_wall(tags, tags_b, [:W, :E, :T, :N, :B], 0, :D)
    FVM.tag_wall(tags, tags_b, :S, 100, :D)
    mesh = FVM.Mesh(volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces, tags, tags_b)
    # Generando los coeficientes
    coeff = FVM.init_coefficients(mesh)
    FVM.set_diffusion(coeff, Γ_constant);
    # Construyendo el sistema de ecuaciones
    equation_system = FVM.init_eq_system(coeff)
    A = equation_system.A
    b = equation_system.b
    return A, b
end

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/oscar/.julia/packages/Plots/9C6z9/src/backends.jl:432


getsystem (generic function with 1 method)

## Pruebas

In [42]:
function medirtiempo(funcion, A, b, mod)
    start_time = time()
    funcion(A, b, mod)
    finish_time = time()
    t = finish_time - start_time
    return t
end

function medirtiempo(funcion, A, b, vector, mod)
    start_time = time()
    funcion(A, b, vector, mod)
    finish_time = time()
    t = finish_time - start_time
    return t
end

medirtiempo (generic function with 2 methods)

In [18]:
A, b = getsystem(40);

In [19]:
@btime x = vectorizedjacobi(A, b);

In [20]:
@btime x = vectorizedgaussseidel(A, b);

In [21]:
α = 1.79
@btime x = vectorizedsor(A, b, α);

In [22]:
@btime x = gmres(A, b);

In [32]:
@btime x = gmresreiniciado(A, b);

In [24]:
M_jacobi = Diagonal(A)
@btime x = gmresprecondicionado(A, b, [M_jacobi]);

In [25]:
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_gauss_seidel_1 = I+(L*inv(D))
M_gauss_seidel_2 = D+U
@btime x = gmresprecondicionado(A, b, [M_gauss_seidel_1, M_gauss_seidel_2]);

In [26]:
α = 1.8
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_sor_1 = I+(α*L*inv(D))
M_sor_2 = D+α*U
@btime x = gmresprecondicionado(A, b, [M_sor_1, M_sor_2]);

In [27]:
M_jacobi = Diagonal(A)
@btime x = gmresprecondicionadoreiniciado(A, b, [M_jacobi]);

  2.263 s (69905 allocations: 12.41 GiB)


In [28]:
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_gauss_seidel_1 = I+(L*inv(D))
M_gauss_seidel_2 = D+U
@btime x = gmresprecondicionadoreiniciado(A, b, [M_gauss_seidel_1, M_gauss_seidel_2]);

  521.051 ms (11929 allocations: 1.47 GiB)


In [29]:
α = 1.8
D = Diagonal(A)
U = triu(A,1) 
L = tril(A,-1)
M_sor_1 = I+(α*L*inv(D))
M_sor_2 = D+α*U
@btime x = gmresprecondicionadoreiniciado(A, b, [M_sor_1, M_sor_2]);

  173.881 ms (3993 allocations: 504.79 MiB)


---

## Sacando los números para el reinicio del gmres

In [62]:
# vols = [10,20,30,40,50,60,80,100]
# modulos = [15,20,25,30,35]
# gmresreiniciado_dict = Dict()
# gmresprecondicionadoreiniciado_jacobi_dict = Dict()
# gmresprecondicionadoreiniciado_gaussseidel_dict = Dict()
# gmresprecondicionadoreiniciado_sor_dict = Dict()
# for vol ∈ vols
#     A, b = getsystem(vol);
#     tiempos_1 = Dict()
#     tiempos_2 = Dict()
#     tiempos_3 = Dict()
#     tiempos_4 = Dict()
#     for modulo ∈ modulos
#         t_1 = medirtiempo(gmresreiniciado, A, b, modulo)
#         tiempos_1[modulo] = t_1
#         M_jacobi = Diagonal(A)
#         t_2 = medirtiempo(gmresprecondicionadoreiniciado, A, b, [M_jacobi], modulo)
#         tiempos_2[modulo] = t_2
#         D = Diagonal(A)
#         U = triu(A,1) 
#         L = tril(A,-1)
#         M_gauss_seidel_1 = I+(L*inv(D))
#         M_gauss_seidel_2 = D+U
#         t_3 = medirtiempo(gmresprecondicionadoreiniciado, A, b, [M_gauss_seidel_1, M_gauss_seidel_2], modulo)
#         tiempos_3[modulo] = t_3
#         α = 1.8
#         D = Diagonal(A)
#         U = triu(A,1) 
#         L = tril(A,-1)
#         M_sor_1 = I+(α*L*inv(D))
#         M_sor_2 = D+α*U
#         t_4 = medirtiempo(gmresprecondicionadoreiniciado, A, b, [M_sor_1, M_sor_2], modulo)
#         tiempos_4[modulo] = t_4
#     end
#     gmresreiniciado_dict[vol] = tiempos_1
#     @show gmresreiniciado_dict[vol]
#     gmresprecondicionadoreiniciado_jacobi_dict[vol] = tiempos_2
#     @show gmresprecondicionadoreiniciado_jacobi_dict[vol]
#     gmresprecondicionadoreiniciado_gaussseidel_dict[vol] = tiempos_3
#     @show gmresprecondicionadoreiniciado_gaussseidel_dict[vol]
#     gmresprecondicionadoreiniciado_sor_dict[vol] = tiempos_4
#     @show gmresprecondicionadoreiniciado_sor_dict[vol]
# end

gmresreiniciado_dict[vol] = Dict{Any, Any}(15 => 0.003920078277587891, 20 => 0.00439000129699707, 25 => 0.004678010940551758, 35 => 0.0053310394287109375, 30 => 0.004033088684082031)
gmresprecondicionadoreiniciado_jacobi_dict[vol] = Dict{Any, Any}(15 => 0.006073951721191406, 20 => 0.022929906845092773, 25 => 0.0063228607177734375, 35 => 0.019644975662231445, 30 => 0.008661031723022461)
gmresprecondicionadoreiniciado_gaussseidel_dict[vol] = Dict{Any, Any}(15 => 0.002471923828125, 20 => 0.0018279552459716797, 25 => 0.002368927001953125, 35 => 0.0041010379791259766, 30 => 0.003072023391723633)
gmresprecondicionadoreiniciado_sor_dict[vol] = Dict{Any, Any}(15 => 0.002460956573486328, 20 => 0.001817941665649414, 25 => 0.0024018287658691406, 35 => 0.003923177719116211, 30 => 0.0030851364135742188)
gmresreiniciado_dict[vol] = Dict{Any, Any}(15 => 0.06982111930847168, 20 => 0.060105085372924805, 25 => 0.06318402290344238, 35 => 0.08980512619018555, 30 => 0.08107113838195801)
gmresprecondicionad

In [71]:
# for key in keys(gmresreiniciado_dict)
#     @show key
#     println(gmresreiniciado_dict[key])
# end

key = 50
Dict{Any, Any}(15 => 4.627002000808716, 20 => 5.456254959106445, 25 => 5.137510061264038, 35 => 5.263601064682007, 30 => 5.207333087921143)
key = 20
Dict{Any, Any}(15 => 0.06982111930847168, 20 => 0.060105085372924805, 25 => 0.06318402290344238, 35 => 0.08980512619018555, 30 => 0.08107113838195801)
key = 10
Dict{Any, Any}(15 => 0.003920078277587891, 20 => 0.00439000129699707, 25 => 0.004678010940551758, 35 => 0.0053310394287109375, 30 => 0.004033088684082031)
key = 60
Dict{Any, Any}(15 => 11.676449060440063, 20 => 12.847965955734253, 25 => 12.274077892303467, 35 => 12.034384965896606, 30 => 12.234254837036133)
key = 30
Dict{Any, Any}(15 => 0.4771289825439453, 20 => 0.39911389350891113, 25 => 0.396104097366333, 35 => 0.38364195823669434, 30 => 0.41845011711120605)
key = 80
Dict{Any, Any}(15 => 52.515424966812134, 20 => 52.741132974624634, 25 => 49.37026596069336, 35 => 49.523401975631714, 30 => 48.8986918926239)
key = 40
Dict{Any, Any}(15 => 1.5194339752197266, 20 => 1.54264783

In [68]:
# for key in keys(gmresprecondicionadoreiniciado_jacobi_dict)
#     @show key
#     println(gmresprecondicionadoreiniciado_jacobi_dict[key])
# end

key = 50
Dict{Any, Any}(15 => 4.627002000808716, 20 => 5.456254959106445, 25 => 5.137510061264038, 35 => 5.263601064682007, 30 => 5.207333087921143)
key = 20
Dict{Any, Any}(15 => 0.06982111930847168, 20 => 0.060105085372924805, 25 => 0.06318402290344238, 35 => 0.08980512619018555, 30 => 0.08107113838195801)
key = 10
Dict{Any, Any}(15 => 0.003920078277587891, 20 => 0.00439000129699707, 25 => 0.004678010940551758, 35 => 0.0053310394287109375, 30 => 0.004033088684082031)
key = 60
Dict{Any, Any}(15 => 11.676449060440063, 20 => 12.847965955734253, 25 => 12.274077892303467, 35 => 12.034384965896606, 30 => 12.234254837036133)
key = 30
Dict{Any, Any}(15 => 0.4771289825439453, 20 => 0.39911389350891113, 25 => 0.396104097366333, 35 => 0.38364195823669434, 30 => 0.41845011711120605)
key = 80
Dict{Any, Any}(15 => 52.515424966812134, 20 => 52.741132974624634, 25 => 49.37026596069336, 35 => 49.523401975631714, 30 => 48.8986918926239)
key = 40
Dict{Any, Any}(15 => 1.5194339752197266, 20 => 1.54264783

In [69]:
# for key in keys(gmresprecondicionadoreiniciado_gaussseidel_dict)
#     @show key
#     println(gmresprecondicionadoreiniciado_gaussseidel_dict[key])
# end

key = 50
Dict{Any, Any}(15 => 1.2286109924316406, 20 => 0.9803910255432129, 25 => 1.2229580879211426, 35 => 1.4405369758605957, 30 => 1.665147066116333)
key = 20
Dict{Any, Any}(15 => 0.024718046188354492, 20 => 0.020373106002807617, 25 => 0.031458139419555664, 35 => 0.020895004272460938, 30 => 0.017122983932495117)
key = 10
Dict{Any, Any}(15 => 0.002471923828125, 20 => 0.0018279552459716797, 25 => 0.002368927001953125, 35 => 0.0041010379791259766, 30 => 0.003072023391723633)
key = 60
Dict{Any, Any}(15 => 2.645148992538452, 20 => 2.9626519680023193, 25 => 2.5288119316101074, 35 => 3.256376028060913, 30 => 2.7802867889404297)
key = 30
Dict{Any, Any}(15 => 0.11976003646850586, 20 => 0.12244415283203125, 25 => 0.10796403884887695, 35 => 0.1751420497894287, 30 => 0.1580212116241455)
key = 80
Dict{Any, Any}(15 => 10.864133834838867, 20 => 10.144203186035156, 25 => 10.261645078659058, 35 => 10.231170177459717, 30 => 9.907202005386353)
key = 40
Dict{Any, Any}(15 => 0.40108489990234375, 20 => 0

In [70]:
# for key in keys(gmresprecondicionadoreiniciado_sor_dict)
#     @show key
#     println(gmresprecondicionadoreiniciado_sor_dict[key])
# end

key = 50
Dict{Any, Any}(15 => 0.4073159694671631, 20 => 0.5515949726104736, 25 => 0.5317399501800537, 35 => 0.42917299270629883, 30 => 0.6807730197906494)
key = 20
Dict{Any, Any}(15 => 0.016109943389892578, 20 => 0.02623605728149414, 25 => 0.016270160675048828, 35 => 0.02901005744934082, 30 => 0.021255016326904297)
key = 10
Dict{Any, Any}(15 => 0.002460956573486328, 20 => 0.001817941665649414, 25 => 0.0024018287658691406, 35 => 0.003923177719116211, 30 => 0.0030851364135742188)
key = 60
Dict{Any, Any}(15 => 1.2782108783721924, 20 => 1.3540899753570557, 25 => 1.3758478164672852, 35 => 1.864130973815918, 30 => 1.3920478820800781)
key = 30
Dict{Any, Any}(15 => 0.058194875717163086, 20 => 0.07891702651977539, 25 => 0.05644392967224121, 35 => 0.0890967845916748, 30 => 0.0719461441040039)
key = 80
Dict{Any, Any}(15 => 3.470571994781494, 20 => 3.7219440937042236, 25 => 3.9240269660949707, 35 => 4.231836795806885, 30 => 3.3793351650238037)
key = 40
Dict{Any, Any}(15 => 0.20071792602539062, 20 